In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("/content/df_clima_Redes.csv")

In [ ]:
data.columns

In [ ]:
X = data[['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'DirVien_ENE',
       'DirVien_ESE', 'DirVien_N', 'DirVien_NE', 'DirVien_NNE', 'DirVien_NNW',
       'DirVien_NW', 'DirVien_S', 'DirVien_SE', 'DirVien_SSE', 'DirVien_SSW',
       'DirVien_SW', 'DirVien_W', 'DirVien_WNW', 'DirVien_WSW',
       'DirVien9am_ENE', 'DirVien9am_ESE', 'DirVien9am_N', 'DirVien9am_NE',
       'DirVien9am_NNE', 'DirVien9am_NNW', 'DirVien9am_NW', 'DirVien9am_S',
       'DirVien9am_SE', 'DirVien9am_SSE', 'DirVien9am_SSW', 'DirVien9am_SW',
       'DirVien9am_W', 'DirVien9am_WNW', 'DirVien9am_WSW',
       'DirVien3am_ENE', 'DirVien3am_ESE', 'DirVien3am_N', 'DirVien3am_NE',
       'DirVien3am_NNE', 'DirVien3am_NNW', 'DirVien3am_NW', 'DirVien3am_S',
       'DirVien3am_SE', 'DirVien3am_SSE', 'DirVien3am_SSW', 'DirVien3am_SW',
       'DirVien3am_W', 'DirVien3am_WNW', 'DirVien3am_WSW',
       'LlueveHoy_Yes',
       'ciudades_Melbourne', 'ciudades_MelbourneAirport', 'ciudades_Sydney',
       'ciudades_SydneyAirport']]
y = data[["LlueveMan_No"]]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
def objective(trial):
    num_layers = trial.suggest_int('num_layers', 1, 3)
    model = Sequential()

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128)
        model.add(Dense(num_units, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()])

    model.fit(X_train_balanced, y_train_balanced, validation_data=(X_valid, y_valid), epochs=5, batch_size=32, verbose=0)

    score = model.evaluate(X_valid, y_valid, verbose=0)
    return score[2]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Mejores parámetros encontrados:", best_params)



[I 2023-11-19 03:45:52,328] A new study created in memory with name: no-name-6b299221-d232-4d27-b225-d3f54fdb16fc
[I 2023-11-19 03:45:58,841] Trial 0 finished with value: 0.8677631616592407 and parameters: {'num_layers': 2, 'n_units_layer_0': 4, 'n_units_layer_1': 49}. Best is trial 0 with value: 0.8677631616592407.
[I 2023-11-19 03:46:12,810] Trial 1 finished with value: 0.8599388599395752 and parameters: {'num_layers': 3, 'n_units_layer_0': 64, 'n_units_layer_1': 7, 'n_units_layer_2': 88}. Best is trial 0 with value: 0.8677631616592407.
[I 2023-11-19 03:46:19,403] Trial 2 finished with value: 0.9259259104728699 and parameters: {'num_layers': 2, 'n_units_layer_0': 46, 'n_units_layer_1': 117}. Best is trial 2 with value: 0.9259259104728699.
[I 2023-11-19 03:46:26,113] Trial 3 finished with value: 0.8606207966804504 and parameters: {'num_layers': 3, 'n_units_layer_0': 27, 'n_units_layer_1': 109, 'n_units_layer_2': 11}. Best is trial 2 with value: 0.9259259104728699.
[I 2023-11-19 03:46:

Mejores parámetros encontrados: {'num_layers': 3, 'n_units_layer_0': 86, 'n_units_layer_1': 54, 'n_units_layer_2': 25}
